# Preparation

In [1]:
# Preparation: Import libraries
%matplotlib inline

import json
import random
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
# Preparation: Load the dataset

def load_jsonl(file_path):
    """Load a JSONL file and return a list of dictionaries."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

# Load train, validation, and test sets
train_data = load_jsonl("data/train_en.jsonl")
val_data = load_jsonl("data/val_en.jsonl")
test_data = load_jsonl("data/test_en.jsonl")

print("Datasets loaded successfully.")

Datasets loaded successfully.


Preview the dataset

In [3]:
# Preview the loaded datasets
print(f"Train Data: {len(train_data)} sentences")
print(f"Validation Data: {len(val_data)} sentences")
print(f"Test Data: {len(test_data)} sentences")

# Preview a sample
print("\nExample from Train Data:")
print(train_data[0])

Train Data: 131280 sentences
Validation Data: 16410 sentences
Test Data: 16454 sentences

Example from Train Data:
{'tokens': ['The', 'type', 'locality', 'is', 'Kīlauea', '.'], 'ner_tags': [0, 0, 0, 0, 5, 0], 'lang': 'en'}


# Data Analasys